# Analyse de sentiments avec Azure

Ici, nous allons utiliser Azure pour faire une analyse de sentiments sur les mêmes données que dans le notebook 3 de ce brief.

In [1]:
import pickle
import string
import pandas as pd

with open("reviews_2", "rb") as pkl_reviews: data = pickle.load(pkl_reviews)

data.head(10)

,target,reviews
0,neg,"plot : two teen couples go to a church party ,..."
1,neg,the happy bastard's quick movie review \ndamn ...
2,neg,it is movies like these that make a jaded movi...
3,neg,""" quest for camelot "" is warner bros . ' firs..."
4,neg,synopsis : a mentally unstable man undergoing ...
5,neg,capsule : in 2176 on the planet mars police ta...
6,neg,"so ask yourself what "" 8mm "" ( "" eight millime..."
7,neg,that's exactly how long the movie felt to me ....
8,neg,call it a road trip for the walking wounded . ...
9,neg,plot : a young french boy sees his parents kil...


In [2]:
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text

def lower_all(text): return text.lower()

data["clean_text"] = data['reviews'].apply(remove_punctuations)
data["clean_text"] = data['clean_text'].apply(lower_all)

In [3]:
# Ici on prépare un échantillon de 10 textes (les 10 premiers du dataset)
# Il faut envoyer à Azure une liste de dictionnaires, d'où le code ci-dessous
text = data.iloc[:10]
reviews = data.iloc[:10]
reviews.drop(columns=["target", "reviews"], inplace=True)
reviews.rename(columns={"clean_text": "text"}, inplace=True)
reviews.reset_index(drop=True, inplace=True)
reviews["id"] = reviews.index
reviews = reviews.to_dict('records')

/tmp/ipykernel_38285/2691663882.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews.drop(columns=["target", "reviews"], inplace=True)
/tmp/ipykernel_38285/2691663882.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews.rename(columns={"clean_text": "text"}, inplace=True)
/tmp/ipykernel_38285/2691663882.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews["id

In [4]:
cog_key = 'd89ce3782b5544a0aa9683d4f72bb915'
cog_endpoint = 'https://azure-ai900-vincent-31012023.cognitiveservices.azure.com/'

print('Ready to use cognitive services at {} using key {}'.format(cog_endpoint, cog_key))

Ready to use cognitive services at https://azure-ai900-vincent-31012023.cognitiveservices.azure.com/ using key d89ce3782b5544a0aa9683d4f72bb915


In [5]:
import os
from azure.cognitiveservices.language.textanalytics import TextAnalyticsClient
from msrest.authentication import CognitiveServicesCredentials

# Get a client for your text analytics cognitive service resource
text_analytics_client = TextAnalyticsClient(endpoint=cog_endpoint,
                                            credentials=CognitiveServicesCredentials(cog_key))


In [6]:
# Use the client and reviews you created previously to get sentiment scores
sentiment_analysis = text_analytics_client.sentiment(documents=reviews)

sentiments = []
for review_num in range(len(reviews)):
    
    
    # Get the sentiment score for this review
    try: sentiment_score = sentiment_analysis.documents[review_num].score
    except IndexError: break

    # classifiy 'positive' if more than 0.5, 
    if sentiment_score < 0.5:
        sentiment = 'negative'
    else:
        sentiment = 'positive'
    
    text_sentiment = {"text": reviews[review_num]["text"],
                     "sentiment": sentiment,
                     "score": sentiment_score}
    
    sentiments.append(text_sentiment)

for s in sentiments:
    print(s["sentiment"])

negative
positive
positive
positive
negative
negative
negative
positive
negative
negative


Même sans calculer de métriques de performances, on constate que les sentiments prédits par Azure diffèrent largement de ceux indiqués dans le dataset. On peut imaginer deux explications à cela :
- Le modèle de prédiction d'Azure n'est pas performant
- Azure Cognitive Services et les personnes qui ont créé le dataset n'ont pas la même définition de ce que le sentiment d'un texte représente. C'est cette explication qui paraît la plus plausible ici.